[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/TripoSR-jupyter/blob/main/TripoSR_texture_gen_jupyter.ipynb)

In [ ]:
%cd /content
!git clone -b dev https://github.com/camenduru/triposr-texture-gen
%cd /content/triposr-texture-gen

!pip install -q accelerate==0.27.2 diffusers==0.26.3 open3d==0.18.0 transformers==4.38.1

import argparse
import copy
import io
import os
import os.path
import shlex
import subprocess
import sys

import numpy as np
import open3d as o3d
import scipy.interpolate
import scipy.spatial
from PIL import Image

def process_tripo_mesh(mesh):
    rot = mesh.get_rotation_matrix_from_xyz((-np.pi / 2, 0, -np.pi / 2))
    new_mesh = copy.deepcopy(mesh)
    new_mesh.rotate(rot)
    new_mesh.remove_non_manifold_edges()
    new_mesh = new_mesh.simplify_quadric_decimation(10000)
    return new_mesh

def raycast_mesh(tmesh):
    scene = o3d.t.geometry.RaycastingScene()
    scene.add_triangles(tmesh)

    rays = scene.create_rays_pinhole(fov_deg=60,
                                     center=[0, 0, 0],
                                     eye=[0, 0, 1.3],
                                     up=[0, -1, 0],
                                     width_px=512,
                                     height_px=512)

    return scene.cast_rays(rays)

def ray_hits_to_depth(raycast_result):
    hits = raycast_result['t_hit'].numpy()
    hits[hits == np.inf] = 0 # REVIEW: can skip for later?

    min1 = np.unique(hits)[1] # min except for 0, mayber better way?
    hits2 = (((np.max(hits) - hits) / (np.max(hits) - min1)).clip(0, 1) * 255).astype('u1')
    hits2[hits == 0] = 0

    return Image.fromarray(hits2)

# - ans_uvs : [N, 2] array
# - ans_prim_ids : [N] array
# - point_colors : [N, 3]
def compute_texture(tmesh, ans_uvs, ans_prim_ids, point_colors, size=512, imdata=None):
    if imdata is None:
        imdata = np.ones((size, size, 3), 'u1') * 255

    # prepend 1 - uv1 - uv2 to make [N, 3] array
    ans_uvs_3 = np.insert(ans_uvs, 0, (1 - np.sum(ans_uvs, 1)), axis=1)

    # Index per-triangle vertex x UVs on triangle IDs from fit to get [N, 3, 2] array
    triuvs = tmesh.triangle.texture_uvs.numpy()[ans_prim_ids]

    # Dot each UV with each triangle UV -> [N, 2] array
    uvs = np.einsum('ij,ijk->ik', ans_uvs_3, triuvs)

    imxy = (uvs * size).astype('u2') # assume size <= max(uint16)

    # interpolate missing pixels...
    interp = scipy.interpolate.LinearNDInterpolator(imxy, point_colors.astype('f4') / 255, 1)
    all_xs_ys = np.indices((size, size)).reshape(2, -1) # [2, N] array
    all_points = all_xs_ys.transpose(1, 0) # [N, 2]

    # ...and mask out points too far from reference points
    kdtree = scipy.spatial.KDTree(imxy)
    dists = kdtree.query(all_points)[0]

    xs, ys = all_xs_ys[:, dists < 2]

    colors = interp(xs, ys)
    imdata[(size - 1) - ys, xs] = colors * 255
    #imdata[(size - 1) - imxy[:, 1], imxy[:, 0]] = point_colors

    return imdata

def compute_raycast_texture(tmesh, raycast_result, rgb_im, size=512):
    print()
    print('computing UV atlas for', len(tmesh.triangle.indices), 'triangles')
    tmesh.compute_uvatlas(size, parallel_partitions=2)
    
    print()
    print('generating texture')
    imdata = tmesh.bake_vertex_attr_textures(size, {'colors'})['colors'].numpy()

    imdata = (imdata * 255).astype('u1')
    prim_ids = raycast_result['primitive_ids'].numpy().flatten()
    mask = prim_ids != 0xffff_ffff
    return compute_texture(
        tmesh,
        raycast_result['primitive_uvs'].numpy().reshape(-1, 2)[mask],
        prim_ids[mask],
        np.array(rgb_im).reshape(-1, 3)[mask],
        size,
        imdata,
    )

def set_tmesh_tex(tmesh, tex_imdata):
    tmesh.material.set_default_properties()
    tmesh.material.material_name = 'defaultLit'
    tmesh.material.texture_maps['albedo'] = o3d.t.geometry.Image(tex_imdata)
    if 'colors' in tmesh.vertex:
        del tmesh.vertex['colors']

def write_mesh(out_base, tmesh):
    out_mesh_path = f'{out_base}.obj'
    o3d.t.io.write_triangle_mesh(out_mesh_path, tmesh)

    # Open 3D seems to have spotty support for writing textures, so manually
    # write out the texture images + update MTL file to reference them
    o3d.t.io.write_image(f'{out_base}.png', tmesh.material.texture_maps['albedo'])
    map_ref_path = os.path.basename(f'{out_base}.png')

    with open(f'{out_base}.mtl', 'a') as mtl_file:
        mtl_file.write(f'\nmap_Ka {map_ref_path}\nmap_Kd {map_ref_path}\n')

In [ ]:
mesh = o3d.io.read_triangle_mesh('/content/burger.obj')
mesh = process_tripo_mesh(mesh)
tmesh = o3d.t.geometry.TriangleMesh.from_legacy(mesh)
raycast_result = raycast_mesh(tmesh)
depth_im = ray_hits_to_depth(raycast_result)
depth_path = f'mesh-preproc-depth.png'
depth_im.save(depth_path)
!python /content/triposr-texture-gen/depth_txt2img.py burger mesh-preproc-depth.png mesh-preproc-depth-paint.png --steps 12 --image-model Lykon/dreamshaper-8 --device cuda:0
tex_imdata = compute_raycast_texture(tmesh, raycast_result, Image.open('/content/mesh-preproc-depth-paint.png'))
set_tmesh_tex(tmesh, tex_imdata)
out_mesh_base = f'new1-tex'
write_mesh(out_mesh_base, tmesh)
tex_imdata